In [29]:
import logging
import os
import pandas as pd
import src.constants as ct


from src.parsedata import PDFParser
from src.initialize import Logger
from src.capitalgains import CapitalGains
from src.utils import rename_mf_scheme


pd.options.display.max_columns = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
!python --version

Python 3.9.5


In [30]:
logger = Logger()
logger.initilaze_logging('no')   
logging.info(f'{"*" * 50}Start Of Process{"*" * 50}')




## Pre-Requisite: CAS statement
The first step is to get the [consolidated account statement](https://www.camsonline.com/Investors/Statements/Consolidated-Account-Statement) from CAMS in PDF Format. Make sure that you select the option as highlighted in yellow.

 <!-- ![screenshot](https://github.com/ajayrawatsap/mf_data/blob/master/data/assets/cams.PNG) -->

 <img src="data/assets/cams.PNG" width=800 height=400 />




## Validate if CAS file exists

Rename your CAS statement from CAMS to *cas.pdf* and place it at root directory. It should be in same directory as this notebook demo.ipynb.
This applies to both local as well as cloud VM enviornment.

In [31]:
import os.path
file_name = 'cas.pdf'
if not os.path.isfile(file_name):
    raise ValueError("file cas.pdf not found in root folder: Hint: Rename your CAS statement file to cas.pdf and save in root directory ")
else: 
    print(f'File {file_name} Found: OK')

File cas.pdf Found: OK


##  Password Input

The password is same as what you set up when requesting the statement from CAMS website.

In [32]:
from getpass import getpass
password = getpass(prompt='Input your password for cas.pdf: ')
# password = tc.password

## Read Contents of PDF

In [33]:
%%time
dp = PDFParser()
    
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_name, password)
print('Parsing CAS statement Data Please wait:', end = " ")


Parsing CAS statement Data Please wait: Wall time: 568 ms


## Calculate LTCG and STCG
By defaut Target LTCG amount is set to INR 100,000. IF you want to set it to different value (e.g.. INR 50,000) change the code in cell below and then run all cells again

target_ltcg = 50000

In [34]:
print('Calculating LTCG and Units to Sell: ', end = " ")

target_ltcg = 100000
cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)
print("Done")

Calculating LTCG and Units to Sell:  Done


## Display Header Data

<BR>The Header and Transaction Data gets saved in data/output directory of Cloud VM or Local Machine as csv files. For Cloud VM Select the checkbox againt each file and download it to your desktops

In [35]:
from src.mfdisplay import get_hdr_data_to_display

hdr_df = get_hdr_data_to_display(cg.output_hdr_df)
hdr_df.head()

,scheme_name,type,latest_nav,latest_nav_date,gf_nav,invested_amt,units,ltcg,stcg,current_amt,perc_gain,target_ltcg,target_units,comments
0,DSP Small Cap Fund - Direct Plan - Growth,EQUITY,109.22,29-Jul-2021,70.16,"₹759,989.98","13,196.65","₹411,306.09","₹91,193.20","₹1,441,351.20",89.65,"₹100,000.00","2,560.30",Sell 2560.295 units for LTCG of 100000.00
1,Franklin India Flexi Cap Fund - Direct Plan - Growth (erstwhile Franklin India Equity Fund - Direct),EQUITY,931.20,29-Jul-2021,637.61,"₹920,001.44","1,667.94","₹529,593.65",₹0.00,"₹1,553,188.70",68.83,"₹100,000.00",340.61,Sell 340.613 units for LTCG of 100000.00
2,Franklin India Low Duration Fund - Direct Plan - Growth,DEBT,24.37,29-Jul-2021,19.98,"₹33,068.34","1,758.42","₹88,730.01","₹-9,984.54","₹42,848.85",29.58,"₹88,730.01","15,952.61",Total LTCG 88730.01 is less than target LTCG 100000: Sell 15952.610 units for LTCG of 88730.01
3,Franklin India Smaller Companies Fund - Direct Plan - Growth,EQUITY,91.42,29-Jul-2021,65.59,"₹696,999.96","12,301.35","₹404,378.97",₹0.00,"₹1,124,617.89",61.35,"₹100,000.00","3,871.12",Sell 3871.123 units for LTCG of 100000.00
4,Franklin India Ultra Short Bond Fund Super Institutional Plan - Direct Plan - Growth,DEBT,30.84,29-Jul-2021,23.80,"₹33,847.65","1,653.00","₹151,858.47","₹-12,680.81","₹50,981.16",50.62,"₹100,000.00","9,647.76",Sell 9647.760 units for LTCG of 100000.00


## Display Transaction Data for Single MF scheme

By default this will show transaction for a random Scheme. You can execute the cell multple times to see results for a diffrent scheme

<br>To view transaction for a specific scheme (e.g. DSP Small Cap Fund - Direct Plan - Growth)  replace  the code in cell to:-
<br> get_trans_for_single_scheme(trans_df,   scheme_name= 'DSP Small Cap Fund - Direct Plan - Growth')

Place cursor on code cell below Hit SHIFT + ENTER to see the results 


#### The *cumil_ltcg* and *cumil_units* columns are useful to know what units to sell for target ltcg.   Sell *cumil_units* for a LTCG of *cumil_ltcg*

In [36]:
from src.mfdisplay import get_trans_data_to_display, get_trans_for_single_scheme
trans_df = get_trans_data_to_display(cg.output_trans_df)

get_trans_for_single_scheme(trans_df, scheme_name= None)


,scheme_name,trans_date,amount,units,purch_nav,trans_type,type,latest_nav,latest_nav_date,gf_nav,new_purch_nav,ltcg,stcg,cumil_ltcg,cumil_units,units_redeemed,units_remain,invested_amt,current_amt,perc_gain
115,ICICI Prudential Long Term Equity Fund (Tax Saving) - Direct Plan - Growth,28-Dec-2018,80000.00,209.35,382.13,PURCHASE,EQUITY,586.23,29-Jul-2021,382.19,382.13,"₹42,728.95",₹0.00,"₹42,728.95",209.35,0,209.353,"₹80,000.06","₹122,729.01",53.41
116,ICICI Prudential Long Term Equity Fund (Tax Saving) - Direct Plan - Growth,10-Jun-2019,70000.00,170.52,410.51,PURCHASE,EQUITY,586.23,29-Jul-2021,382.19,410.51,"₹29,963.77",₹0.00,"₹72,692.72",379.87,0,170.520,"₹70,000.17","₹99,963.94",42.80


## Portfolio Summary

In [37]:
from src.mfdisplay import group_by_mf_type, display_summary
mf_totals_grp = group_by_mf_type(cg.mf_hdr_df)
display_summary(mf_totals_grp)



            

##  Equiy vs Debt Aggregate Data

In [38]:
from src.mfdisplay import display_equity_vs_debt
display_equity_vs_debt(mf_totals_grp)

## Plot Mutual Funds Scheme Amounts

In [39]:
from src.mfdisplay import display_mf_scheme_amounts


display_mf_scheme_amounts(cg.mf_hdr_df)                 

## Plot Unrealised LTCG and STCG

In [40]:
from src.mfdisplay import display_unrealised_ltcg_stcg
display_unrealised_ltcg_stcg(cg.mf_hdr_df)               

## Plot Mutual Funds Allocation by Scheme

In [41]:
from src.mfdisplay import display_mf_scheme_pie_chart

display_mf_scheme_pie_chart(cg.mf_hdr_df)

In [42]:

logging.info(f'{"*" * 50}End Of Process{"*" * 50}\n') 